In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from keras.datasets import cifar10
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
subset_size = 1000  
x_train = x_train[:subset_size]
y_train = y_train[:subset_size]

In [5]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [7]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(kernel='rbf', probability=True, random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=5)

In [8]:
rf_clf.fit(x_train, y_train)
svm_clf.fit(x_train, y_train)
knn_clf.fit(x_train, y_train)

KNeighborsClassifier()

In [9]:
y_pred_rf = rf_clf.predict(x_val)
y_pred_svm = svm_clf.predict(x_val)
y_pred_knn = knn_clf.predict(x_val)

In [10]:
def evaluate_individual_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy of {model_name} Model: {accuracy}")
    conf_mat = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix for {model_name} Model:\n{conf_mat}")
    class_report = classification_report(y_true, y_pred)
    print(f"Classification Report for {model_name} Model:\n{class_report}")

In [11]:
evaluate_individual_model(y_val, y_pred_rf, "Random Forest")
evaluate_individual_model(y_val, y_pred_svm, "SVM")
evaluate_individual_model(y_val, y_pred_knn, "KNN")

Accuracy of Random Forest Model: 0.355
Confusion Matrix for Random Forest Model:
[[11  1  0  1  0  0  1  0  5  0]
 [ 0 12  0  0  1  1  3  1  4  8]
 [ 2  1  6  1  3  2  5  1  0  2]
 [ 2  1  1  3  2  1  4  2  1  3]
 [ 1  0  1  0  7  2  6  4  2  0]
 [ 1  0  2  0  1  3  3  2  0  0]
 [ 0  1  5  1  2  1  7  3  0  0]
 [ 1  0  2  0  6  1  1  4  0  1]
 [ 2  0  0  0  1  2  0  2 11  1]
 [ 0  2  0  0  1  0  4  3  1  7]]
Classification Report for Random Forest Model:
              precision    recall  f1-score   support

           0       0.55      0.58      0.56        19
           1       0.67      0.40      0.50        30
           2       0.35      0.26      0.30        23
           3       0.50      0.15      0.23        20
           4       0.29      0.30      0.30        23
           5       0.23      0.25      0.24        12
           6       0.21      0.35      0.26        20
           7       0.18      0.25      0.21        16
           8       0.46      0.58      0.51        19


## Max Voting Ensemble Model

In [12]:
voting_clf = VotingClassifier(estimators=[('rf', rf_clf), ('svm', svm_clf), ('knn', knn_clf)], voting='soft')

In [13]:
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svm', SVC(probability=True, random_state=42)),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')

In [14]:
y_pred = voting_clf.predict(x_val)

In [15]:
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy of Ensemble Classifier:", accuracy)
conf_mat = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:")
print(conf_mat)
class_report = classification_report(y_val, y_pred)
print("Classification Report:")
print(class_report)

Validation Accuracy of Ensemble Classifier: 0.285
Confusion Matrix:
[[ 9  0  2  1  2  0  0  0  5  0]
 [ 2  8  2  0  0  0  5  1  5  7]
 [ 5  0  5  0 10  0  3  0  0  0]
 [ 3  1  1  2  3  1  4  2  2  1]
 [ 1  1  2  0  7  1  5  5  1  0]
 [ 1  0  2  0  2  2  4  1  0  0]
 [ 0  0  6  0  6  1  6  1  0  0]
 [ 3  0  3  0  4  0  2  3  0  1]
 [ 4  0  1  1  0  0  1  1 10  1]
 [ 0  1  1  2  0  0  4  2  3  5]]
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.47      0.38        19
           1       0.73      0.27      0.39        30
           2       0.20      0.22      0.21        23
           3       0.33      0.10      0.15        20
           4       0.21      0.30      0.25        23
           5       0.40      0.17      0.24        12
           6       0.18      0.30      0.22        20
           7       0.19      0.19      0.19        16
           8       0.38      0.53      0.44        19
           9       0.33      0.28    

## Averaging Ensemble Model

In [16]:
y_pred_rf_proba = rf_clf.predict_proba(x_val)
y_pred_svm_proba = svm_clf.predict_proba(x_val)
y_pred_knn_proba = knn_clf.predict_proba(x_val)
avg_pred_proba = (y_pred_rf_proba + y_pred_svm_proba + y_pred_knn_proba) / 3
y_pred_avg = np.argmax(avg_pred_proba, axis=1)

In [17]:
accuracy_avg = accuracy_score(y_val, y_pred_avg)
print(f"Accuracy of Averaging Ensemble Model: {accuracy_avg}")
conf_mat_avg = confusion_matrix(y_val, y_pred_avg)
print(f"Confusion Matrix for Averaging Ensemble Model:\n{conf_mat_avg}")
class_report_avg = classification_report(y_val, y_pred_avg)
print(f"Classification Report for Averaging Ensemble Model:\n{class_report_avg}")

Accuracy of Averaging Ensemble Model: 0.285
Confusion Matrix for Averaging Ensemble Model:
[[ 9  0  2  1  2  0  0  0  5  0]
 [ 2  8  2  0  0  0  5  1  5  7]
 [ 5  0  5  0 10  0  3  0  0  0]
 [ 3  1  1  2  3  1  4  2  2  1]
 [ 1  1  2  0  7  1  5  5  1  0]
 [ 1  0  2  0  2  2  4  1  0  0]
 [ 0  0  6  0  6  1  6  1  0  0]
 [ 3  0  3  0  4  0  2  3  0  1]
 [ 4  0  1  1  0  0  1  1 10  1]
 [ 0  1  1  2  0  0  4  2  3  5]]
Classification Report for Averaging Ensemble Model:
              precision    recall  f1-score   support

           0       0.32      0.47      0.38        19
           1       0.73      0.27      0.39        30
           2       0.20      0.22      0.21        23
           3       0.33      0.10      0.15        20
           4       0.21      0.30      0.25        23
           5       0.40      0.17      0.24        12
           6       0.18      0.30      0.22        20
           7       0.19      0.19      0.19        16
           8       0.38      0.53      

## Weighted Averaging Ensemble Model


In [18]:
rf_weight = 0.4
svm_weight = 0.3
knn_weight = 0.3
weighted_avg_pred_proba = (rf_weight * y_pred_rf_proba + svm_weight * y_pred_svm_proba + knn_weight * y_pred_knn_proba)
y_pred_weighted_avg = np.argmax(weighted_avg_pred_proba, axis=1)

In [19]:
accuracy_weighted_avg = accuracy_score(y_val, y_pred_weighted_avg)
print(f"Accuracy of Weighted Averaging Ensemble Model: {accuracy_weighted_avg}")
conf_mat_weighted_avg = confusion_matrix(y_val, y_pred_weighted_avg)
print(f"Confusion Matrix for Weighted Averaging Ensemble Model:\n{conf_mat_weighted_avg}")
class_report_weighted_avg = classification_report(y_val, y_pred_weighted_avg)
print(f"Classification Report for Weighted Averaging Ensemble Model:\n{class_report_weighted_avg}")

Accuracy of Weighted Averaging Ensemble Model: 0.285
Confusion Matrix for Weighted Averaging Ensemble Model:
[[ 9  1  1  1  1  0  1  0  5  0]
 [ 1  8  2  0  0  1  5  1  5  7]
 [ 5  0  5  0 10  0  3  0  0  0]
 [ 3  1  1  2  3  1  4  2  2  1]
 [ 1  1  2  0  7  1  5  5  1  0]
 [ 1  0  2  0  2  2  4  1  0  0]
 [ 0  0  6  0  6  1  6  1  0  0]
 [ 3  0  3  0  4  0  2  3  0  1]
 [ 4  0  1  1  0  0  1  1 10  1]
 [ 0  1  1  2  0  0  4  2  3  5]]
Classification Report for Weighted Averaging Ensemble Model:
              precision    recall  f1-score   support

           0       0.33      0.47      0.39        19
           1       0.67      0.27      0.38        30
           2       0.21      0.22      0.21        23
           3       0.33      0.10      0.15        20
           4       0.21      0.30      0.25        23
           5       0.33      0.17      0.22        12
           6       0.17      0.30      0.22        20
           7       0.19      0.19      0.19        16
           8